<td>
<a href="https://colab.research.google.com/github/raoulg/MADS-DAV/blob/main/notebooks/06.2-modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</td>

In [ ]:
import requests
from pathlib import Path
import pandas as pd
import seaborn as sns
from loguru import logger

This notebook is an implementation of the approach described in this [essay](https://www.orphanalytics.com/en/news/whitepaper202012/OrphAnalyticsQAnon2020.pdf) and written about [here](https://www.prnewswire.com/news-releases/qanon-is-two-different-people-shows-machine-learning-analysis-from-orphanalytics-301192981.html).

It analyzes the posts of QAnon, and concludes that the messages are probably written by two persons instead of one.

From the last link:
>  QAnon has spread conspiracy theories to an unprecedentedly large audience. Its thousands of online messages have popularized narratives such as the existence of a child-trafficking deep state. Recently, it inspired a series of violent attacks and was listed as a terrorist threat by the FBI. The Swiss company OrphAnalytics just published an analysis of all messages posted by Q. Its patented technology aims at identifying authors of written documents. It has found two individual signals within the corpus of Q messages. Its new study contributes to revealing the origins and the persons behind one of the most impactful conspiracy theories in recent times.

We will replicate their analysis with "patented technoloy" with a simple sklearn approach.

In [ ]:
datadir = Path.home() / ".cache/mads_datasets/qanon"
if not datadir.exists():
    datadir.mkdir(parents=True)

datafile = datadir / "posts.json"
if not datafile.exists():
    logger.info("Downloading QAnon data")
    url = "https://raw.githubusercontent.com/jkingsman/JSON-QAnon/main/posts.json"
    response = requests.get(url, timeout=10)
    with datafile.open("wb") as f:
        f.write(response.content)
else:
    logger.info(f"QAnon data already downloaded at {datafile}")

On Linux and macOS, the default encoding is typically UTF-8.
On Windows, the default encoding is not UTF-8 but rather a locale-specific encoding, often CP1252 or similar. Because, why not use your own proprietary encoding, just to be backwards compatible, even though the rest of the world uses something different?

If you are running this code on a windows machine, you will need to specify the encoding as 'utf-8', like I did below:

In [ ]:
from pandas import json_normalize
import json

with datafile.open(encoding="utf-8") as f:
    df = json_normalize(json.load(f)["posts"], sep="_")

In [ ]:
df.dtypes

## Clean up the timestamps

The text describes they used the analysis from [this](https://www.businessinsider.com/every-qanon-message-q-drop-analyzed-2020-10?international=true&r=US&IR=T) businessinsider article to bin the timeseries.

<img src="../img/Qanon.png" width=450 height=400 />

These dates are hardcoded in the bin_time method.

In [ ]:
df["time"] = df["post_metadata_time"].apply(pd.to_datetime, unit="s")

In [ ]:
from datetime import datetime


def bin_time(time):
    if time < datetime(2017, 12, 1):
        return 0
    elif time < datetime(2018, 1, 6):
        return 1
    elif time < datetime(2018, 8, 10):
        return 2
    elif time < datetime(2019, 8, 1):
        return 3
    else:
        return 4


df["bintime"] = df["time"].apply(lambda x: bin_time(x))

In [ ]:
from datetime import timezone
datetime(2017, 12, 1, tzinfo=timezone.utc)

In [ ]:
df["bintime"].value_counts()

Let's clean up the text some more. We will replace linebreaks ("\n") with spaces, remove urls and set everything to lowercase.

In [ ]:
import re


def remove_url(text):
    return re.sub(r"^https?:\/\/.*[\r\n]*", "", text)


df["text"] = df["text"].apply(lambda x: str(x).replace("\n", " "))
df["text"] = df["text"].apply(lambda x: remove_url(x))
df["text"] = df["text"].apply(lambda x: x.lower())

Now, we count every text and filter chuncks with size below 50.

In [ ]:
df["size"] = df["text"].apply(lambda x: len(str(x)))
df = df[df["size"] > 50]
df.reset_index(inplace=True, drop=True)

The paper describes batching all the text into chunks of 7500 characters. We can obtain sort of the same by splitting the text up into 100 batches, 7.7k each.

In [ ]:
def batch_seq(text, k):
    longseq = " ".join(text)
    n = int(len(longseq) / k)
    parts = [longseq[i : i + n] for i in range(0, len(longseq), n)]
    return parts, n


z = 100
parts, n = batch_seq(df["text"], k=z)
parts = parts[:-1]
len(parts), n

In [ ]:
# This gives us groups of 100 posts
df["size"].cumsum() // n

In [ ]:
# Lets store that group in the dataframe
df.loc[:, "group"] = df["size"].cumsum() // n

The paper uses a ngram with size 3 on characterlevel. We can do that with `CountVectorizer`

How does this work? We will get a matrix with 100 rows (one for every group) and many columns. What are these columns? This are the trigram-counts. Let's see how this works: for example, if I have the string "yellow banana", we get the trigrams: " y", "ye", "el", "ll", "lo", "ow", "w ", " b", "ba", "an", "na", "an", "na". We can see that the trigram "an" appears twice. So, the column "an" will have the value 2, the other trigrams have the value 1. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

example = ["yellow banana", "papagena papaya banana"]
vectorizer = CountVectorizer(analyzer="char", ngram_range=(3, 3))
trigram_example = vectorizer.fit_transform(example)
logger.info(f"The trigrams are: {vectorizer.get_feature_names_out()}")
logger.info(f"Shape of the trigram matrix: {trigram_example.shape}")
trigram_example.toarray()

In this example, we see the model counted 23 different trigrams.

Now let's proceed to do the same with our text.

In [ ]:
import numpy as np

vectorizer = CountVectorizer(analyzer="char", ngram_range=(3, 3))
X = vectorizer.fit_transform(parts)
X = np.asarray(X.todense())
X.shape

You can see we found 23074 different trigrams in our text.

We can now calculates the pairwise manhattan distances. That way, we get an idea how how different every chunk is, when compared to all other chunks, in terms of the ngrams. If the vectors are similar, the distance is zero, if they are very different, the distance is high.

This will give us a 100x100 matrix with the distances, for every vector the distance to all other vectors.

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances

distance = manhattan_distances(X, X)
distance.shape, type(distance)

Lets check how this works for the trigram example:

In [ ]:
manhattan_distances(trigram_example.toarray(), trigram_example.toarray())

Here we see that the manhattan distance between the trigram vectors of "yellow banana" and "papagena papaya banana" is 21.
To handle the labels, we make integers of the labels.

In [ ]:
labels = df["post_metadata_source_site"].unique()
mapping = {labels[i]: i for i in range(3)}
df["source"] = df["post_metadata_source_site"].apply(lambda x: mapping[x])
source_names = {v: k for k, v in mapping.items()}
source_names

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
model = pca.fit_transform(distance)

In [ ]:
import matplotlib.pyplot as plt

sourcemean = np.round(df.groupby("group").source.mean()).values.reshape(-1, 1)
timemean = np.round(df.groupby("group").bintime.mean()).values.reshape(-1, 1)
p = pd.DataFrame(
    np.concatenate([model, sourcemean, timemean], axis=1),
    columns=["x", "y", "source", "time"],
).reset_index()
p["source"] = p["source"].map(source_names)
sns.scatterplot(data=p, x="x", y="y", hue="source", style="source", palette="Set1")
plt.yticks([])
plt.xticks([])
plt.title("QAnon posts show two distinct authors")
plt.legend(title="Forum source")

This reproduces the main conclusion, that the red source is not overlapping with the other two sources.

Of course, this tells you nothing about how high the chance is, that this is actually caused by having two authors, or by somehow changing the writing style.

For that, we would need access to how likely it is that the writing style changes this much.

However, we still have reproduced their "patented technology" with a few lines of code.
Let's wrap it into a class:

In [ ]:
import sys

sys.path.insert(0, "..")

from wa_analyzer.model import TextClustering

clustering = TextClustering()
labels = sourcemean.flatten().astype(str)
clustering(text=df["text"], k=100, labels=labels, batch=True, method="PCA")

And now lets test it on the whatsapp data:

In [ ]:
import tomllib

configfile = Path("../config.toml").resolve()
with configfile.open("rb") as f:
    config = tomllib.load(f)
datafile = (Path("..") / Path(config["processed"]) / config["current"]).resolve()
if not datafile.exists():
    logger.warning(
        "Datafile does not exist. First run src/preprocess.py, and check the timestamp!"
    )
wa_df = pd.read_parquet(datafile)
wa_df.head()

We know the authors, so let's use the labels.

In [ ]:
authors = list(np.unique(wa_df.author))

We take a subset of every author, and join their texts.
Obviously, you can only do this if you know the authors, or at least the claimed authors. Concatenating all texts together will make the data useless. 

In [ ]:
n = 500
min_parts = 2

corpus = {}
for author in authors:
    subset = wa_df[wa_df.author == author].reset_index()
    longseq = " ".join(subset.message)
    # chunk everything into n-sized parts
    parts = [longseq[i : i + n] for i in range(0, len(longseq), n)]
    # clean urls
    parts = [remove_url(chunk) for chunk in parts]
    # remove double spaces
    parts = [re.sub(" +", " ", chunk) for chunk in parts]
    # keep only parts with more than min_parts
    if len(parts) > min_parts:
        corpus[author] = parts
corpus.keys()

In [ ]:
import matplotlib.pyplot as plt

text = [part for text in corpus.values() for part in text]
wa_labels = [k for k, v in corpus.items() for _ in range(len(v))]
# we set batch to false, because we already batched the data
clustering(text=text, k=100, labels=wa_labels, batch=False, method="tSNE")
plt.legend(title="Author", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xticks([])
plt.yticks([])
plt.title("Distinct authors in the WhatsApp dataset")